날짜 변환<br/>
결측 값 행 제거 + 결측 값 많은 열 제거 + 수치 데이터 정규화 작업<br/>
10 단위 1로 수정<br/>
null data 분리<br/>
ATA 년도, 월만 추가<br/>
PORT_SIZE LabelEncofing<br/>

In [10]:
import warnings
warnings.filterwarnings(action="ignore")
import datetime
import time
import bisect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
train_df = pd.read_parquet('../../data/HD_data/train.parquet').drop(columns=['SAMPLE_ID'])
test_df = pd.read_parquet('../../data/HD_data/test.parquet').drop(columns=['SAMPLE_ID'])

In [3]:
# datetime 컬럼 처리
train_df['ATA'] = pd.to_datetime(train_df['ATA'])
test_df['ATA'] = pd.to_datetime(test_df['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train_df, test_df]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month

# datetime 컬럼 제거
train_df.drop(columns=['ATA'], inplace=True)
test_df.drop(columns=['ATA'], inplace=True)

In [4]:
scaler = StandardScaler()
x = train_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH"]]
scaler.fit(x)
train_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH"]] = scaler.transform(x)
test_feautre = test_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH"]]
test_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH"]] = scaler.transform(test_feautre)

In [5]:
# train, test 동일한 경우
train_df = pd.get_dummies(train_df, columns=["ARI_CO", "ARI_PO", "SHIP_TYPE_CATEGORY"], drop_first=True)
test_df = pd.get_dummies(test_df, columns=["ARI_CO", "ARI_PO", "SHIP_TYPE_CATEGORY"], drop_first=True)

In [6]:
for name in ["BREADTH", "DEPTH", "DRAUGHT"]:
    train_df[name] = train_df[name].apply(lambda x: x // 10)
    test_df[name] = test_df[name].apply(lambda x: x // 10)

In [7]:
train_df = train_df.dropna(subset=["BREADTH", "DEPTH", "DRAUGHT", "LENGTH"], axis=0)

In [8]:
train_dummies = pd.get_dummies(train_df, columns=["FLAG"])
test_dummies = pd.get_dummies(test_df, columns=["FLAG"])

train_df, test_df = train_dummies.align(test_dummies, axis=1, fill_value=0)

In [9]:
assert sorted(list(train_df.columns)) == sorted(list(test_df.columns))

In [11]:
categorical_features = ['ID', 'SHIPMANAGER', 'PORT_SIZE']
encoders = {}

for feature in categorical_features:
  le = LabelEncoder()
  train_df[feature] = le.fit_transform(train_df[feature].astype(str))
  le_classes_set = set(le.classes_)
  test_df[feature] = test_df[feature].map(lambda s: '-1' if s not in le_classes_set else s)
  le_classes = le.classes_.tolist()
  bisect.insort_left(le_classes, '-1')
  le.classes_ = np.array(le_classes)
  test_df[feature] = le.transform(test_df[feature].astype(str))
  encoders[feature] = le

In [12]:
remain_idx = train_df[train_df["U_WIND"].notna() & train_df["V_WIND"].notna() & train_df["AIR_TEMPERATURE"].notna() & train_df["BN"].notna()].index
train_remain_df = train_df.loc[remain_idx]

In [13]:
remain_idx = test_df[test_df["U_WIND"].notna() & test_df["V_WIND"].notna() & test_df["AIR_TEMPERATURE"].notna() & test_df["BN"].notna()].index
test_remain_df = test_df.loc[remain_idx]

In [ ]:
train_df.drop(columns=["U_WIND", "V_WIND", "AIR_TEMPERATURE", "BN"], inplace=True)
test_df.drop(columns=["U_WIND", "V_WIND", "AIR_TEMPERATURE", "BN"], inplace=True)

In [14]:
train_df.to_parquet("../../data/HD_data/train_v2_remove.parquet")
train_remain_df.to_parquet("../../data/HD_data/train_v2_remain.parquet")
test_df.to_parquet("../../data/HD_data/test_v2_remove.parquet")
test_remain_df.to_parquet("../../data/HD_data/test_v2_remain.parquet")